<a href="https://colab.research.google.com/github/johyunkang/python_stat/blob/main/chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12 회귀분석

## 12.4 모형의 타당성

**모형의 타당성** 이란? 

오차항 $\epsilon_i $는 서로 독립이고 $N(0, \sigma^2)$을 따른다. 라는 가정을 만족하고 있는지 여부를 체크하는 것

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/누구나파이썬통계분석/data/ch12_scores_reg.csv')
n = len(df)
print('n:', n)
print('shape:', df.shape)
display(df.describe())
display(df.head())

n: 20
shape: (20, 4)


,quiz,final_test,sleep_time
count,20.000000,20.000000,20.000000
mean,3.685000,47.850000,6.210000
std,2.487923,19.826285,1.337673
min,0.000000,18.000000,4.100000
25%,1.800000,35.000000,5.150000
50%,3.750000,43.500000,6.750000
75%,5.450000,61.750000,7.350000
max,8.800000,88.000000,7.900000


,quiz,final_test,sleep_time,school_method
0,4.2,67,7.2,bus
1,7.2,71,7.9,bicycle
2,0.0,19,5.3,bus
3,3.0,35,6.8,walk
4,1.5,35,7.5,walk


In [ ]:
formula = 'final_test ~ quiz + sleep_time'
result = smf.ols(formula, df).fit()
display(result.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             final_test   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     26.35
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           6.19e-06
Time:                        14:51:56   Log-Likelihood:                -73.497
No. Observations:                  20   AIC:                             153.0
Df Residuals:                      17   BIC:                             156.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8709     11.635     -0.161      0.874     -26.420      22.678
quiz           6.4289      0.956      6.725      0.000       4.412       8.446
sleep_time     4.1917      1.778      2.357      0.031       0.440       7.943
==============================================================================
Omnibus:                        2.073   Durbin-Watson:                   1.508
Prob(Omnibus):                  0.355   Jarque-Bera (JB):                1.716
Skew:                           0.660   Prob(JB):                        0.424
Kurtosis:                       2.437   Cond. No.                         38.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
eps_hat = np.array(result.resid) # 잔차 (epsilon)

### 12.4.1 정규성 검정

오차항 $\epsilon_i$ 가 $N(0, \sigma^2)$을 따른다는 가정이 타당했는지 알아보기 위해, 잔차가 **정규분포**를 따르고 있는지를 확인하는 정규성 검정을 수행

정규성 검정으로 `Omnibus` 검정과 `Jarque-Bera (JB)` 검정이 사용됨

- 귀무가설 : 잔차항은 정규분포를 따릅니다.
- 대립가설 : 잔차항은 정규분포를 따르지 않습니다.


이러한 검정의 p값인 `Prob(Omnibus)`나 `Prob(JB)`가 **유의수준**인 0.05 보다 크면 문제될게 없음

왜도(Skew)와 첨도(Kurtosis)를 살펴보는 것으로도 정규성을 확인할 수 있음
- 왜도 : 좌우대칭. 좌우대칭이면 0, 오른쪽으로 치우친 분포면 0보다 작은 값
    - `stats.skew`로 계산 가능

    - $\displaystyle \sum_i^n \left( \dfrac {x - \bar{x}} {S} \right)^3$
- 첨도 : 볼록한 정도
    - 정규분포이면 3, 정규분포보다 뾰족하면 3보다 큼. 3보다 작으면 더 낮은 정점
    - $\displaystyle \sum_i^n \left( \dfrac {x - \bar{x}} {S} \right)^4$

In [9]:
# 왜도(skew) 
# 왜도 > 0 : 왼쪽으로 치우침
# 왜도 < 0 : 오른쪽으로 치우침
print('왜도(skew):', stats.skew(eps_hat))

# fisher=False : 정규분포의 첨도=3 이 되는 것을 의미. (Pearson 분포를 따른다)
# fisher=True  : 정규분포의 첨도=0 이되는 것을 의미.
print('첨도(kurtosis):', stats.kurtosis(eps_hat, fisher=False)) 

왜도(skew): 0.6599489486177468
첨도(kurtosis): 2.436583325676301


### 12.4.2 더빈왓슨비

더빈-왓슨비 (Durbin-Watson ratio) : 다른 오차항이 서로 무상관인지 여부를 체크하는 지표

시계열 데이터인 경우 특히 중요한 데이터

$\dfrac {\displaystyle \sum_{i=2}^n (\hat{\epsilon_i} - \hat{\epsilon}_{i-1})^2} {\displaystyle \sum_{i=1}^n \hat{\epsilon}_i^2 }$

더빈-왓슨비 : 0부터 4의 값이
 - 양의상관 : 0에 가까우면
 - 음의상관 : 4에 가까우면
 - 무상관 :  2 앞뒤의 값

In [10]:
print('더빈-왓슨비:', np.sum(np.diff(eps_hat, 1)**2) / np.sum(eps_hat ** 2))

더빈-왓슨비: 1.5082185264423016


### 12.4.3 다중공선성

`Cond. No.` : 조건수라고 부르는 것이므로 **다중공선성**을 체크하는 지표

조건수의 값이 크면 다중공선성과 설명변수 사이에 강한 상관이 생겼다는 것을 의미

다중공선성이 크면 회귀계수의 분산이 커져 모형의 예측 결과가 나빠진다고 알려져 있음


---


아래 테스트로 쪽지시험의 결과를 2배로 한 중간고사라는 변수를 생성. 당연하지만 쪽지시험과 중간고사의 상관관계는 1이 되고 큰 다중공선성을 가짐

In [11]:
df['mid_test'] = df['quiz'] * 2
display(df.head())

,quiz,final_test,sleep_time,school_method,mid_test
0,4.2,67,7.2,bus,8.4
1,7.2,71,7.9,bicycle,14.4
2,0.0,19,5.3,bus,0.0
3,3.0,35,6.8,walk,6.0
4,1.5,35,7.5,walk,3.0


In [12]:
formula = 'final_test ~ quiz + mid_test'
result = smf.ols(formula, df).fit()
result.summary()
# summary 가장 아래의 Cond.No.(조건수) 가 1.09e + 17로 매우 큰 값이 나옴
# 다중공선성이 생기면 조건수는 매우 큰 값이 됨.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             final_test   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     37.61
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           8.59e-06
Time:                        15:23:54   Log-Likelihood:                -76.325
No. Observations:                  20   AIC:                             156.7
Df Residuals:                      18   BIC:                             158.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     23.6995      4.714      5.028      0.000      13.796      33.603
quiz           1.3107      0.214      6.133      0.000       0.862       1.760
mid_test       2.6215      0.427      6.133      0.000       1.723       3.519
==============================================================================
Omnibus:                        2.139   Durbin-Watson:                   1.478
Prob(Omnibus):                  0.343   Jarque-Bera (JB):                1.773
Skew:                           0.670   Prob(JB):                        0.412
Kurtosis:                       2.422   Cond. No.                     1.09e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.65e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""